# Дашборд для Яндекс.Дзен.

<h1>Оглавление:<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Техническое-задание:" data-toc-modified-id="Техническое-задание:-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Техническое задание:</a></span></li><li><span><a href="#Вопросы-менеджеров:" data-toc-modified-id="Вопросы-менеджеров:-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Вопросы менеджеров:</a></span></li></ul></li><li><span><a href="#Шаг-1.-Импортирую-необходимые-библиотеки,-открываю-файл-с-данными-и-изучаю-общую-информацию." data-toc-modified-id="Шаг-1.-Импортирую-необходимые-библиотеки,-открываю-файл-с-данными-и-изучаю-общую-информацию.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Шаг 1. Импортирую необходимые библиотеки, открываю файл с данными и изучаю общую информацию.</a></span><ul class="toc-item"><li><span><a href="#Подключаюсь-к-базе-данных." data-toc-modified-id="Подключаюсь-к-базе-данных.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подключаюсь к базе данных.</a></span></li><li><span><a href="#Пишу-SQL-запрос,-для-получения-всех-данных,-сохраняю-данные-в-dash_visits." data-toc-modified-id="Пишу-SQL-запрос,-для-получения-всех-данных,-сохраняю-данные-в-dash_visits.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Пишу SQL-запрос, для получения всех данных, сохраняю данные в <code>dash_visits</code>.</a></span></li><li><span><a href="#Изучаю-общую-информацию." data-toc-modified-id="Изучаю-общую-информацию.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Изучаю общую информацию.</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Шаг-2.-Сохраняю-датасет-в-CSV-файл." data-toc-modified-id="Шаг-2.-Сохраняю-датасет-в-CSV-файл.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Шаг 2. Сохраняю датасет в CSV файл.</a></span></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общий вывод:</a></span></li></ul></div>

### Техническое задание:

- Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
- Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
- Кто будет основным пользователем дашборда: менеджеры по анализу контента;

- Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;
    
- По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;
    
- Характер данных:
    - История событий по темам карточек - абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников - относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
    
- Важность: все графики имеют равную важность;
- Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица `log_raw`);
- База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД `zen`;
- Частота обновления данных: один раз в сутки, в полночь по UTC;
- Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (уточнено при создании макета дашборда).

### Вопросы менеджеров:
- Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

## Шаг 1. Импортирую необходимые библиотеки, открываю файл с данными и изучаю общую информацию.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

### Подключаюсь к базе данных.

In [2]:
db_config = {
    'user':'praktikum_student',
    'pwd':'Sdf4$2;d-d30pp',
    'host':'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port':6432,
    'db':'data-analyst-zen-project-db'
} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)
engine = create_engine(connection_string) 

### Пишу SQL-запрос, для получения всех данных, сохраняю данные в `dash_visits`.

In [3]:
query = '''
        SELECT *
        FROM dash_visits
        '''

In [4]:
dash_visits = pd.io.sql.read_sql(query, con = engine)

### Изучаю общую информацию.

In [5]:
def df_info(df):
    display(df.head(3))
    print('-' * 50)
    df.info()
    print('-' * 50)
    display(df.describe(include = 'all'))
    print('-' * 50)
    print('Количество явных дубликатов:', df.duplicated().sum())

In [6]:
df_info(dash_visits)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
--------------------------------------------------


,record_id,item_topic,source_topic,age_segment,dt,visits
count,3.074500e+04,30745,30745,30745,30745,30745.000000
unique,NaN,25,26,6,17,NaN
top,NaN,Отношения,Семейные отношения,18-25,2019-09-24 18:58:00,NaN
freq,NaN,1536,1822,7056,3383,NaN
first,NaN,NaN,NaN,NaN,2019-09-24 18:28:00,NaN
last,NaN,NaN,NaN,NaN,2019-09-24 19:00:00,NaN
mean,1.055969e+06,NaN,NaN,NaN,NaN,10.089673
std,8.875461e+03,NaN,NaN,NaN,NaN,19.727601
min,1.040597e+06,NaN,NaN,NaN,NaN,1.000000
25%,1.048283e+06,NaN,NaN,NaN,NaN,1.000000


--------------------------------------------------
Количество явных дубликатов: 0


### Вывод:
Данные прекрасны, пропусков и дубликатов нет, типы данных столбцов соответствуют содержимому.
## Шаг 2. Сохраняю датасет в CSV файл.

In [7]:
dash_visits.to_csv('dash_visits.csv')

## Общий вывод:
Работу над этим проектом я провел на удаленной машине в сервисе Yandex.Cloud. Мной был установлен PostgreSQL и развернута база данных. Затем я написал скрипт пайплайна, который позволил собирать данные за определенный временной период, и настроил его автономную работу через crontab. Для визуализации собранных данных я написал скрипт дашборда с несколькими фильтрами и также запустил его на удаленной машине. По результатам была подготовлена презентация с полученными графиками. 
***
Всего, за изучаемый период, в системе с разбивкой по темам карточек происходит **310207** взаимодействий пользователей с карточками.

Больше всего посещений у следующих тем источников:
- `Семейные отношения` – **33309**;
- `Россия` - **29831**;
- `Полезные советы` - **27412**;
- `Путешествия` - **24124**;
- `Знаменитости` - **23945**;
- `Кино` - **20084**;
- `Дети` – **15243**.

Составить топ тем карточек на каждую конкретную минуту с разбивками по темам и возрастным группам, можно используя необходимые фильтры в самом дашборде:

[Дашборд](https://public.tableau.com/app/profile/evgeny.babaevsky/viz/Dashbord_for_Yandeks_Dzen/Dashboard1?publish=yes), [Презентация](https://disk.yandex.ru/i/waZRx9l2AeX0zw).